In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import shutil

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 1 if using cudnn on GPU else using CPU
CUDNN_GPU = 0
model_path = 'model.xlsx'
data_path = '../data/LRCN_dataset.npy'
model_saver = 'model_saver/'
n_classes = 4
n_input = [224,224,3]
n_output = [4]
batch_size = 5

In [3]:
data = np.load(data_path).item()
#data['X_train'] = data['X_train']/255
my_model = pd.read_excel(model_path)
num_layer = my_model.values.shape[0]
my_model

,Order,Layer,Filter,kernel,Stride,Padding,Output
0,0,Input,NaN,NaN,NaN,NaN,224x224x3
1,1,Conv,96.0,7x7,2.0,VALID,110x110x96
2,2,MaxPooling,NaN,3x3,2.0,NaN,55x55x96
3,3,Conv,256.0,5x5,2.0,VALID,26x26x256
4,4,MaxPooling,NaN,3x3,2.0,NaN,13x13x256
5,5,Conv,384.0,3x3,1.0,SAME,13x13x384
6,6,Conv,384.0,3x3,1.0,SAME,13x13x384
7,7,Conv,256.0,3x3,1.0,SAME,13x13x256
8,8,MaxPooling,NaN,3x3,2.0,NaN,6x6x256
9,9,Flatten,NaN,NaN,NaN,NaN,9216x1


In [4]:
def loss_op(model, y):
    model = tf.Print(model,[model],summarize=20,message="model")
    y = tf.Print(y,[y],summarize=20,message="y____")
    
#     loss =  tf.reduce_sum(tf.square(model - y),[1])
#     return tf.reduce_mean(loss)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model,labels= y))

In [5]:

use_cudnn_on_gpu = True if CUDNN_GPU == 1 else False
my_model = pd.read_excel(model_path)

### Define some function for making layers
def make_input(input_, out_shape, name):
    #return tf.reshape(input_, [1,out_shape[0],out_shape[1],out_shape[2]], name=name)
    result = input_
    tf.summary.histogram(name, result)
    return result

def make_conv(input_, in_channel, out_channel, filter_, strides, padding_,  name):
    out_channel = int(out_channel)
    strides = int(strides)
    filter_ = list(map(int,filter_.split('x')))
    filter_ = tf.Variable(tf.random_normal([filter_[0],filter_[1],in_channel,out_channel],stddev=0.01), trainable=True, name=name+'_filter')
    # conv
    result = tf.nn.conv2d(input=input_, 
                     filter=filter_,
                     strides=[1, strides, strides, 1],
                     padding=padding_,
                     use_cudnn_on_gpu=use_cudnn_on_gpu,
                     name=name)
    # add bias
    bias = tf.Variable(tf.random_normal([out_channel]))
    result = tf.nn.bias_add(result, bias)
    # relu
    result = tf.nn.relu(result)

    tf.summary.histogram(name, result)
    return result

def make_maxpool(input_, in_channel, filter_, strides, name):
    strides = int(strides)
    filter_ = list(map(int,filter_.split('x')))
    result = tf.nn.max_pool(value=input_,
                         ksize=[1, filter_[0], filter_[1], 1],
                         strides=[1, strides, strides, 1],
                         padding='SAME',
                         name=name)
    return result

def make_flatten(input_, name):
    #input_ = tf.Print(input_,[input_],summarize=20,message='flatten')
    return tf.contrib.layers.flatten(inputs=input_, scope=name)

def make_fc(input_, out_shape, name):
    result = tf.contrib.layers.fully_connected(inputs=input_,
                                                activation_fn=tf.nn.sigmoid,
                                                num_outputs=int(out_shape[0]),
                                                scope=name)
    #result = tf.layers.dense(input_, units=int(out_shape[0]), name=name)
    tf.summary.histogram(name, result)
    return result

def make_dropout(input_, name):
    return tf.nn.dropout(input_, 0.75)

def make_reshape(input_, out_shape, name):
    result = tf.reshape(tensor=input_,
                     shape=[tf.shape(input_)[0],out_shape[0],out_shape[1],out_shape[2]],
                     name=name)
    tf.summary.histogram(name, result)
    return result

def make_sigmoid(input_, name):
    return tf.nn.sigmoid(input_, name=name)

def make_softmax(input_, name):
    #input_ = tf.Print(input_,[input_],summarize=20,message='Before softmax')
    return tf.nn.softmax(input_, name=name)
    #return input_

### Generate the model base on the model file
#output.append(input)

layer_match = {
        'Input':      lambda input_, params, _:       make_input(input_, params[-1], 'input_image'),
        'Conv':       lambda input_, params, channel: make_conv(input_, channel, params[2], params[3], params[4], str(params[5]), 'layer_'+str(params[0])),
        'MaxPooling': lambda input_, params, channel: make_maxpool(input_, channel, params[3], params[4], 'layer_'+str(params[0])),
        'Flatten':    lambda input_, params, _:       make_flatten(input_, 'layer_'+str(params[0])),
        'Fc':         lambda input_, params, _:       make_fc(input_, params[-1], 'layer_'+str(params[0])),
        'Reshape':    lambda input_, params, _:       make_reshape(input_, params[-1], 'layer_'+str(params[0])),
        'Dropout':    lambda input_, params, _:       make_dropout(input_, 'layer_'+str(params[0])),
        'Sigmoid':    lambda input_, params, _:       make_sigmoid(input_, 'layer_'+str(params[0])),
        'Softmax':    lambda input_, params, _:       make_softmax(input_, 'layer_'+str(params[0]))
    }
prev_channel = None




#print(output.shape)


In [6]:
train_len = data['X_train'].shape[0]
test_len = data['X_test'].shape[0]
print(train_len, test_len)

1742 333


In [7]:
data['y_train'].shape

(1742, 4)

In [8]:
# reset graph
tf.reset_default_graph()

# the placeholders will be used for the feed_dict param of tf.Session.run()
is_training = tf.placeholder(tf.bool, name='is_training')
X = tf.placeholder(tf.float32, [None, n_input[0], n_input[1], n_input[2]], name='X_train')
y = tf.placeholder(tf.float32, [None, n_output[0]], name='y_train')

In [9]:
def cnn_model(input, is_training=True):
    # model
    #cnn_model = cnn_model(X, model_path, is_training)
    input2 = make_input(input,[224,224,3],'input')
    conv1 = make_conv(input2,3,96,'7x7',2,'VALID','conv1')
    pool1 = make_maxpool(conv1,96,'3x3',2,'pool1')
    conv2 = make_conv(pool1,96,256,'5x5',2,'VALID','conv2')
    pool2 = make_maxpool(conv2,256,'3x3',2,'pool2')
    conv3 = make_conv(pool2,256,384,'3x3',1,'SAME','conv3')
    conv4 = make_conv(conv3,384,384,'3x3',1,'SAME','conv4')
    conv5 = make_conv(conv4,384,256,'3x3',1,'SAME','conv5')
    pool5 = make_maxpool(conv5,256,'3x3',2,'pool5')
    flatten = make_flatten(pool5,'flatten')
    fc1 = make_fc(flatten,[4096,1],'fc1')
    fc2 = make_fc(fc1,[4096,1],'fc2')
    fc3 = make_fc(fc2,[4,1],'fc3')
    softmax = make_softmax(fc3,'softmax')
    
    return softmax

In [10]:

cnn_model1 = cnn_model(X)

# learning rate
global_step = tf.Variable(0, trainable=False, name='global_step')
# learning_rate = tf.train.exponential_decay(
#                         0.001,  # Base learning rate.
#                         global_step,  # Current index into the dataset.
#                         train_len,  # Decay step.
#                         0.95,  # Decay rate.
#                         staircase=True,
#                         name='learning_rate')
learning_rate = 0.001



# loss 
loss_op = loss_op(cnn_model1, y);
# optimizer 
learning_rate = tf.Print(learning_rate,[learning_rate],message=">>>>>>")
optimal = tf.train.AdamOptimizer(learning_rate, name='adam_optimizer')
#optimal = tf.train.MomentumOptimizer(learning_rate, 0.9)
# increment global_step at each step.
train_op = optimal.minimize(loss_op, global_step=global_step, name='optimal_min')

# evaluate model
correct_prediction = tf.equal(tf.argmax(cnn_model1, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
# tao summary cua cac monitor de quan sat cac bien
tf.summary.scalar('loss_op', loss_op)
tf.summary.scalar('learning_rate', learning_rate)
tf.summary.scalar('accuracy', accuracy)

# gop cac summaries vao mot operation
merged_summary_op = tf.summary.merge_all()

# tao doi tuong log writer va ghi vao Tensorboard
tf_writer = tf.summary.FileWriter('../checkpoint', graph=tf.get_default_graph())

# khoi tao cac variables
init = tf.global_variables_initializer()
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [12]:
def get_batch(dict_field_name, batch):
    return data[dict_field_name][batch*batch_size:min((batch+1)*batch_size,train_len)]

In [13]:
# remove model_saver folder
shutil.rmtree(model_saver, ignore_errors=True)

# training
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init, feed_dict={is_training:True})
    # Training cycle
    epoch = 0;
    while True:
        total_batch =  train_len // batch_size
        #for batch in range(total_batch):
        for batch in range(5):
            # lay batch tiep theo
            batch_input = get_batch('X_train', batch) 
            batch_label = get_batch('y_train', batch)
            
            
            
            # chay train_op, loss_op, accuracy
            _, cost, acc, summary = sess.run([train_op, loss_op, accuracy, merged_summary_op], feed_dict={X:batch_input, y:batch_label, is_training:True})
            # Write logs at every iteration
            tf_writer.add_summary(summary, epoch * total_batch + batch)
            #print("Epoch:" + ('%04d,' % (epoch)) + ("cost={%.9f}, training accuracy %.5f" % (cost, acc)) + "\n")
            
            

        epoch += 1;
        
        # hien thi ket qua sau moi epoch
        print("Epoch:" + ('%04d,' % (epoch)) + ("cost={%.9f}, training accuracy %.5f" % (cost, acc)) + "\n")
        
        if epoch % 10 == 0:
            # Luu tru variables vao disk.
            save_path = saver.save(sess, model_saver + 'nn_model_%04d.ckpt'%(epoch))
            print("Model saved in path: %s \n" % save_path)

Epoch:0001,cost={1.369679213}, training accuracy 0.60000

Epoch:0002,cost={1.369844794}, training accuracy 0.60000



KeyboardInterrupt: 